In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 40),
            classifier_block(40, 30),
            classifier_block(30, 25),
            classifier_block(25, 20),
            nn.Linear(20, 18)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_13 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_14 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_15 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_16 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_17 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_18 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12) + [12] * len(X_13) + [13] * len(X_14) + [14] * len(X_15) + [15] * len(X_16) + [16] * len(X_17) + [17] * len(X_18)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 11.486918687820435, Final Batch Loss: 2.798377275466919
Epoch 2, Loss: 11.599191904067993, Final Batch Loss: 2.9199044704437256
Epoch 3, Loss: 11.635527610778809, Final Batch Loss: 2.9617674350738525
Epoch 4, Loss: 11.411921262741089, Final Batch Loss: 2.746338367462158
Epoch 5, Loss: 11.50093126296997, Final Batch Loss: 2.846264123916626
Epoch 6, Loss: 11.637593984603882, Final Batch Loss: 2.9973459243774414
Epoch 7, Loss: 11.63011884689331, Final Batch Loss: 2.990257978439331
Epoch 8, Loss: 11.442826747894287, Final Batch Loss: 2.8147552013397217
Epoch 9, Loss: 11.402079343795776, Final Batch Loss: 2.780205726623535
Epoch 10, Loss: 11.511808395385742, Final Batch Loss: 2.905625820159912
Epoch 11, Loss: 11.507941961288452, Final Batch Loss: 2.9171841144561768
Epoch 12, Loss: 11.445812940597534, Final Batch Loss: 2.87876558303833
Epoch 13, Loss: 11.36445689201355, Final Batch Loss: 2.822486162185669
Epoch 14, Loss: 11.456492185592651, Final Batch Loss: 2.942516326904297


Epoch 115, Loss: 7.2396416664123535, Final Batch Loss: 1.5786840915679932
Epoch 116, Loss: 7.486827492713928, Final Batch Loss: 1.866039514541626
Epoch 117, Loss: 7.19476854801178, Final Batch Loss: 1.5948172807693481
Epoch 118, Loss: 7.391346216201782, Final Batch Loss: 1.7725478410720825
Epoch 119, Loss: 7.1940836906433105, Final Batch Loss: 1.5019824504852295
Epoch 120, Loss: 6.965549349784851, Final Batch Loss: 1.3292608261108398
Epoch 121, Loss: 6.796924591064453, Final Batch Loss: 1.1219438314437866
Epoch 122, Loss: 6.946574926376343, Final Batch Loss: 1.2762773036956787
Epoch 123, Loss: 7.292706847190857, Final Batch Loss: 1.6512038707733154
Epoch 124, Loss: 6.354308903217316, Final Batch Loss: 0.8046358227729797
Epoch 125, Loss: 7.070627331733704, Final Batch Loss: 1.4095733165740967
Epoch 126, Loss: 7.710625648498535, Final Batch Loss: 1.9397917985916138
Epoch 127, Loss: 6.353235900402069, Final Batch Loss: 0.5350474715232849
Epoch 128, Loss: 7.784826755523682, Final Batch Los

Epoch 235, Loss: 6.68075966835022, Final Batch Loss: 1.5962300300598145
Epoch 236, Loss: 5.947876513004303, Final Batch Loss: 0.9293424487113953
Epoch 237, Loss: 5.620170474052429, Final Batch Loss: 0.614837646484375
Epoch 238, Loss: 6.640827775001526, Final Batch Loss: 1.595555305480957
Epoch 239, Loss: 7.081352472305298, Final Batch Loss: 2.1042845249176025
Epoch 240, Loss: 6.387154936790466, Final Batch Loss: 1.4415926933288574
Epoch 241, Loss: 6.3840168714523315, Final Batch Loss: 1.4844155311584473
Epoch 242, Loss: 5.189683020114899, Final Batch Loss: 0.20042544603347778
Epoch 243, Loss: 5.713385164737701, Final Batch Loss: 0.8102836012840271
Epoch 244, Loss: 5.828915894031525, Final Batch Loss: 0.8962194323539734
Epoch 245, Loss: 6.15729546546936, Final Batch Loss: 1.1651731729507446
Epoch 246, Loss: 7.124658107757568, Final Batch Loss: 2.1449666023254395
Epoch 247, Loss: 6.077902436256409, Final Batch Loss: 1.1694879531860352
Epoch 248, Loss: 7.078617334365845, Final Batch Loss:

Epoch 355, Loss: 5.793393015861511, Final Batch Loss: 1.3220722675323486
Epoch 356, Loss: 5.916965126991272, Final Batch Loss: 1.1316766738891602
Epoch 357, Loss: 7.581018328666687, Final Batch Loss: 2.9031522274017334
Epoch 358, Loss: 6.0466402769088745, Final Batch Loss: 1.5415936708450317
Epoch 359, Loss: 7.022238731384277, Final Batch Loss: 2.4649133682250977
Epoch 360, Loss: 5.676608204841614, Final Batch Loss: 1.2806627750396729
Epoch 361, Loss: 6.655315279960632, Final Batch Loss: 2.109907865524292
Epoch 362, Loss: 6.463378548622131, Final Batch Loss: 1.8324031829833984
Epoch 363, Loss: 5.8807350397109985, Final Batch Loss: 1.302881121635437
Epoch 364, Loss: 5.474005103111267, Final Batch Loss: 0.867477297782898
Epoch 365, Loss: 5.896258473396301, Final Batch Loss: 1.4634393453598022
Epoch 366, Loss: 5.898759365081787, Final Batch Loss: 1.5066261291503906
Epoch 367, Loss: 6.1246994733810425, Final Batch Loss: 1.7544528245925903
Epoch 368, Loss: 4.394489474594593, Final Batch Los

Epoch 469, Loss: 5.099093377590179, Final Batch Loss: 0.9346840977668762
Epoch 470, Loss: 5.967965364456177, Final Batch Loss: 1.8493486642837524
Epoch 471, Loss: 4.9720951318740845, Final Batch Loss: 0.8654394149780273
Epoch 472, Loss: 5.587938189506531, Final Batch Loss: 1.554125189781189
Epoch 473, Loss: 6.188748598098755, Final Batch Loss: 2.068449020385742
Epoch 474, Loss: 5.363922595977783, Final Batch Loss: 1.3762527704238892
Epoch 475, Loss: 5.0501062870025635, Final Batch Loss: 1.010066270828247
Epoch 476, Loss: 5.45414924621582, Final Batch Loss: 1.4579862356185913
Epoch 477, Loss: 5.578767657279968, Final Batch Loss: 1.5176451206207275
Epoch 478, Loss: 5.458387017250061, Final Batch Loss: 1.4896526336669922
Epoch 479, Loss: 5.634007930755615, Final Batch Loss: 1.5802116394042969
Epoch 480, Loss: 6.42603075504303, Final Batch Loss: 2.43857741355896
Epoch 481, Loss: 4.139693811535835, Final Batch Loss: 0.07877428829669952
Epoch 482, Loss: 4.314639985561371, Final Batch Loss: 0

...

Epoch 4491, Loss: 3.192118465900421, Final Batch Loss: 1.7135509252548218
Epoch 4492, Loss: 2.8179002702236176, Final Batch Loss: 1.3376483917236328
Epoch 4493, Loss: 3.2016141414642334, Final Batch Loss: 1.5594792366027832
Epoch 4494, Loss: 6.616860628128052, Final Batch Loss: 4.885774612426758
Epoch 4495, Loss: 1.6904127324232832, Final Batch Loss: 0.0005414212355390191
Epoch 4496, Loss: 2.437986671924591, Final Batch Loss: 0.8003668189048767
Epoch 4497, Loss: 5.527565598487854, Final Batch Loss: 3.798023223876953
Epoch 4498, Loss: 1.6961387479677796, Final Batch Loss: 0.006947644986212254
Epoch 4499, Loss: 1.6681605763733387, Final Batch Loss: 0.0008927173912525177
Epoch 4500, Loss: 1.6807885766029358, Final Batch Loss: 0.07806900143623352
Epoch 4501, Loss: 2.885831117630005, Final Batch Loss: 1.1926515102386475
Epoch 4502, Loss: 1.8236435241997242, Final Batch Loss: 0.05068828538060188
Epoch 4503, Loss: 2.0407804548740387, Final Batch Loss: 0.38013938069343567
Epoch 4504, Loss: 1.8

Epoch 4607, Loss: 1.577736258506775, Final Batch Loss: 0.04944407939910889
Epoch 4608, Loss: 2.5305623412132263, Final Batch Loss: 1.0359145402908325
Epoch 4609, Loss: 1.4783464600332081, Final Batch Loss: 0.002816759515553713
Epoch 4610, Loss: 1.910438358783722, Final Batch Loss: 0.3666794002056122
Epoch 4611, Loss: 1.4952558623626828, Final Batch Loss: 0.010122393257915974
Epoch 4612, Loss: 1.5762798264622688, Final Batch Loss: 0.12119264155626297
Epoch 4613, Loss: 1.5477404571138322, Final Batch Loss: 0.0038890219293534756
Epoch 4614, Loss: 1.8319577872753143, Final Batch Loss: 0.26949170231819153
Epoch 4615, Loss: 1.5745856016874313, Final Batch Loss: 0.1103726178407669
Epoch 4616, Loss: 1.4710652868961915, Final Batch Loss: 0.0008407871937379241
Epoch 4617, Loss: 1.5371640287339687, Final Batch Loss: 0.023129668086767197
Epoch 4618, Loss: 2.1025468707084656, Final Batch Loss: 0.6343157887458801
Epoch 4619, Loss: 1.8827941119670868, Final Batch Loss: 0.38258296251296997
Epoch 4620,

Epoch 4722, Loss: 2.291076123714447, Final Batch Loss: 0.8346859216690063
Epoch 4723, Loss: 1.6142405718564987, Final Batch Loss: 0.1275591105222702
Epoch 4724, Loss: 2.0922585129737854, Final Batch Loss: 0.6077709197998047
Epoch 4725, Loss: 1.520387778058648, Final Batch Loss: 0.010613778606057167
Epoch 4726, Loss: 1.6883578896522522, Final Batch Loss: 0.15205705165863037
Epoch 4727, Loss: 3.375294119119644, Final Batch Loss: 1.875974416732788
Epoch 4728, Loss: 1.460058628454135, Final Batch Loss: 4.60137271147687e-05
Epoch 4729, Loss: 1.5572859607636929, Final Batch Loss: 0.014582600444555283
Epoch 4730, Loss: 1.443637311457337, Final Batch Loss: 1.5497195136049413e-06
Epoch 4731, Loss: 1.467833032249473, Final Batch Loss: 0.0018799504032358527
Epoch 4732, Loss: 1.777338445186615, Final Batch Loss: 0.26549863815307617
Epoch 4733, Loss: 1.4532524733513128, Final Batch Loss: 0.0004698126285802573
Epoch 4734, Loss: 1.6852223873138428, Final Batch Loss: 0.29233962297439575
Epoch 4735, Lo

Epoch 4830, Loss: 3.4072801172733307, Final Batch Loss: 1.8561813831329346
Epoch 4831, Loss: 3.3889695405960083, Final Batch Loss: 1.7825924158096313
Epoch 4832, Loss: 3.371537208557129, Final Batch Loss: 1.8015174865722656
Epoch 4833, Loss: 3.0101715326309204, Final Batch Loss: 1.5109624862670898
Epoch 4834, Loss: 1.6869443580508232, Final Batch Loss: 0.0653320774435997
Epoch 4835, Loss: 1.620162027888, Final Batch Loss: 0.011286932043731213
Epoch 4836, Loss: 2.5057454705238342, Final Batch Loss: 0.9250546097755432
Epoch 4837, Loss: 1.5484149150215671, Final Batch Loss: 3.9457496313843876e-05
Epoch 4838, Loss: 1.719757780432701, Final Batch Loss: 0.1959162801504135
Epoch 4839, Loss: 1.5932591408491135, Final Batch Loss: 0.0667712539434433
Epoch 4840, Loss: 3.62074613571167, Final Batch Loss: 2.1628530025482178
Epoch 4841, Loss: 1.509004186897073, Final Batch Loss: 0.0008322112844325602
Epoch 4842, Loss: 2.0483208894729614, Final Batch Loss: 0.5309142470359802
Epoch 4843, Loss: 1.68318

Epoch 4949, Loss: 1.4214582731947303, Final Batch Loss: 0.00042500998824834824
Epoch 4950, Loss: 1.4079046892002225, Final Batch Loss: 0.01009890902787447
Epoch 4951, Loss: 1.7672795057296753, Final Batch Loss: 0.3139195740222931
Epoch 4952, Loss: 1.8681931793689728, Final Batch Loss: 0.448967307806015
Epoch 4953, Loss: 2.8853592574596405, Final Batch Loss: 1.4647200107574463
Epoch 4954, Loss: 1.4399963913547253, Final Batch Loss: 1.8954096958623268e-05
Epoch 4955, Loss: 1.5250357426702976, Final Batch Loss: 0.03468722477555275
Epoch 4956, Loss: 1.434776388690807, Final Batch Loss: 0.0018941095331683755
Epoch 4957, Loss: 1.408200915902853, Final Batch Loss: 0.02883654460310936
Epoch 4958, Loss: 2.4705729484558105, Final Batch Loss: 0.9617026448249817
Epoch 4959, Loss: 2.144141048192978, Final Batch Loss: 0.7060433030128479
Epoch 4960, Loss: 3.233532190322876, Final Batch Loss: 1.8377058506011963
Epoch 4961, Loss: 1.6053512143553235, Final Batch Loss: 0.0003313469351269305
Epoch 4962, L

In [9]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[27  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  4  0  0  0  0  0  3  0  0  1  0  0  2]
 [ 0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  4  0  0  2  0  0  2  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  1  0  0 14  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  7  0  0  0  0]
 [ 0  0  3  0  0  1  0  0  2  0  0  0  0  0  3  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  0]
 [ 0  0  3  0  0  0  0  0  0  0

# Train on Fake Test on Real

In [10]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [11]:
gen = Generator(z_dim = 109)
load_model(gen, "3 Label 6 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=109, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=33, bias=True)
    (4): Tanh()
  )
)

In [37]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 6)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 3:
        fake_labels.append(9)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 3:
        fake_labels.append(10)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 3:
        fake_labels.append(11)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 4:
        fake_labels.append(12)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 4:
        fake_labels.append(13)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 4:
        fake_labels.append(14)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 5:
        fake_labels.append(15)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 5:
        fake_labels.append(16)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 5:
        fake_labels.append(17)
        
fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [38]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5, zero_division = 0))

[[11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0  0  0  5  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  7  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  4  0  0  0  0  0  6  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0  0  0  0  0